In [ ]:
import triqstools.tools as tools
import triqstools.pytools as pytools
from triqstools.pytools import print_mpi
from pytriqs.gf import Gf, BlockGf, iOmega_n, Idx
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, MeshImTime, MeshProduct, MeshCyclicLattice, MeshPoint
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.archive import HDFArchive, hdf_archive_schemes
from pytriqs.utility import mpi
from pytriqs.lattice.tight_binding import TightBinding, dos
from triqs_ctint import SolverCore
import numpy as np
from math import pi
from math import sqrt
from scipy.optimize import curve_fit, approx_fprime
from scipy import polyfit, polyval, polyder

from pytriqs.plot.mpl_interface import oplot, oploti, oplotr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
get_ipython().magic(u'matplotlib inline')
matplotlib.rcParams['lines.linewidth'] = 2
matplotlib.rcParams['font.size'] = 48
import h5py
from numpy import matrix
%matplotlib inline

In [ ]:
input_file = "weak_coupling_fluct_diag.h5"

In [ ]:
with HDFArchive(input_file, 'r') as A:
    beta     = A['beta']
    n_k      = A['n_k']
    k_mesh   = A['k_mesh']
    n_q      = A['n_q']
    q_mesh   = A['q_mesh']
    Sigma    = A['Sigma']
    Sigma_q  = A['Sigma_q']
    Sigma_kp = A['Sigma_kp']

In [ ]:
oploti(Sigma((pi,0,0),all), '-o', label=r'$\mathbf{k}$=AN', color='red', x_window=(0,60/beta))
oploti(Sigma((pi/2,pi/2,0),all), '-v', label=r'$\mathbf{k}$=N', color='blue', x_window=(0,60/beta))

plt.legend(loc=2, fontsize=14)
plt.xlabel(r'$\omega_{n}$', fontsize=18)
plt.ylabel(r'Im $\Sigma(\mathbf{k}, i\omega_{n})$', fontsize=16)
plt.show()

In [ ]:
k_lin = np.linspace(-pi, pi, n_k, endpoint=True)
kx, ky = np.meshgrid(k_lin, k_lin)

self_plot = lambda kx, ky: Sigma((kx,ky,0),0).imag

plt.figure(figsize=(8,8))
gs=plt.GridSpec(2,2)
ax = plt.subplot(gs[1],aspect="equal")
plt.pcolormesh(kx, ky, np.vectorize(self_plot)(kx,ky))
plt.colorbar(shrink=0.8)

plt.axes
ax.set_yticks([-np.pi,-np.pi/2.,0,np.pi/2,np.pi])
ax.set_yticklabels([r"$-\pi$",r"$-\frac{\pi}{2}$","0",r"$\frac{\pi}{2}$",r"$\pi$"],fontsize=16)
ax.set_xticks([-np.pi,-np.pi/2.,0,np.pi/2,np.pi])
ax.set_xticklabels([r"$-\pi$",r"$-\frac{\pi}{2}$","0",r"$\frac{\pi}{2}$",r"$\pi$"],fontsize=16)
plt.tight_layout(pad=0,w_pad=2,h_pad=0)
plt.title(r'Im $\Sigma(\mathbf{k},i\omega_{0})$',size=20)
plt.show()

In [ ]:
oploti(Sigma_q[:,Idx(k_mesh.locate_neighbours((pi,0,0))),:]((pi,pi,0),all), '-o', label=r'$\mathbf{q}=(\pi,\pi)$', color='red', x_window=(0,60/beta))
oploti(Sigma_q[:,Idx(k_mesh.locate_neighbours((pi,0,0))),:]((0,0,0),all), '-v', label=r'$\mathbf{q}=(0,0)$', color='blue', x_window=(0,60/beta))

plt.legend(loc=2, fontsize=14)
plt.xlabel(r'$\omega_{n}$', fontsize=18)
plt.ylabel(r'Im $\tilde{\Sigma}_{\mathbf{q}}(\mathbf{k}=(\pi,0), i\omega_{n})$', fontsize=16)
plt.show()

In [ ]:
q_lin = np.linspace(0., 2.*pi, n_q, endpoint=True)
qx, qy = np.meshgrid(q_lin, q_lin)

self_plot = lambda qx, qy: Sigma_q[:,Idx(k_mesh.locate_neighbours((pi,0,0))),:]((qx,qy,0),0).imag

plt.figure(figsize=(8,8))
gs=plt.GridSpec(2,2)
ax = plt.subplot(gs[1],aspect="equal")
plt.pcolormesh(qx, qy, np.vectorize(self_plot)(qx,qy))
plt.colorbar(shrink=0.8)

plt.axes
plt.xlabel(r"$q_{x}$", fontsize=18)
plt.ylabel(r"$q_{y}$", fontsize=18)
ax.set_yticks([0.,np.pi/2.,np.pi,3*np.pi/2.,2*np.pi])
ax.set_yticklabels(["0",r"$\frac{\pi}{2}$",r"$\pi$",r"$\frac{3\pi}{2}$","$2\pi$"],fontsize=16)
ax.set_xticks([0.,np.pi/2.,np.pi,3*np.pi/2.,2*np.pi])
ax.set_xticklabels(["0",r"$\frac{\pi}{2}$",r"$\pi$",r"$\frac{3\pi}{2}$","$2\pi$"],fontsize=16)
plt.tight_layout(pad=0,w_pad=2,h_pad=0)
plt.title(r'Im $\tilde{\Sigma}_{\mathbf{q}}(\mathbf{k}=(\pi,0), i\omega_{0})$',size=20)
plt.show()

In [ ]:
oploti(Sigma_kp[:,Idx(k_mesh.locate_neighbours((pi,0,0))),:]((pi,0,0),all), '-o', label=r"$\mathbf{k'}=(\pi,0)$", color='red', x_window=(0,60/beta))
oploti(Sigma_kp[:,Idx(k_mesh.locate_neighbours((pi,0,0))),:]((pi/2,pi/2,0),all), '-v', label=r"$\mathbf{k'}=(\pi/2,\pi/2)$", color='blue', x_window=(0,60/beta))

plt.legend(loc=4, fontsize=14)
plt.xlabel(r'$\omega_{n}$', fontsize=18)
plt.ylabel(r"Im $\tilde{\Sigma}_{\mathbf{k'}}(\mathbf{k}=(\pi,0), i\omega_{n})$", fontsize=16)
plt.show()

In [ ]:
k_lin = np.linspace(-pi, pi, n_k, endpoint=True)
kx, ky = np.meshgrid(k_lin, k_lin)

self_plot = lambda kx, ky: Sigma_kp[:,Idx(k_mesh.locate_neighbours((pi,0,0))),:]((kx,ky,0),0).imag

plt.figure(figsize=(8,8))
gs=plt.GridSpec(2,2)
ax = plt.subplot(gs[1],aspect="equal")
plt.pcolormesh(kx, ky, np.vectorize(self_plot)(kx,ky))
plt.colorbar(shrink=0.8)

plt.axes
plt.xlabel(r"$k'_{x}$", fontsize=18)
plt.ylabel(r"$k'_{y}$", fontsize=18)
ax.set_yticks([-np.pi,-np.pi/2.,0,np.pi/2,np.pi])
ax.set_yticklabels([r"$-\pi$",r"$-\frac{\pi}{2}$","0",r"$\frac{\pi}{2}$",r"$\pi$"],fontsize=16)
ax.set_xticks([-np.pi,-np.pi/2.,0,np.pi/2,np.pi])
ax.set_xticklabels([r"$-\pi$",r"$-\frac{\pi}{2}$","0",r"$\frac{\pi}{2}$",r"$\pi$"],fontsize=16)
plt.tight_layout(pad=0,w_pad=2,h_pad=0)
plt.title(r"Im $\tilde{\Sigma}_{\mathbf{k'}}(\mathbf{k}=(\pi,0), i\omega_{0})$",size=20)
plt.show()

In [ ]:
oploti(Sigma_kp[:,Idx(k_mesh.locate_neighbours((pi/2,pi/2,0))),:]((pi,0,0),all), '-o', label=r"$\mathbf{k'}=(\pi,0)$", color='red', x_window=(0,60/beta))
oploti(Sigma_kp[:,Idx(k_mesh.locate_neighbours((pi/2,pi/2,0))),:]((pi/2,pi/2,0),all), '-v', label=r"$\mathbf{k'}=(\pi/2,\pi/2)$", color='blue', x_window=(0,60/beta))

plt.legend(loc=4, fontsize=14)
plt.xlabel(r'$\omega_{n}$', fontsize=18)
plt.ylabel(r"Im $\tilde{\Sigma}_{\mathbf{k'}}(\mathbf{k}=(\pi/2,\pi/2), i\omega_{n})$", fontsize=16)
plt.show()

In [ ]:
k_lin = np.linspace(-pi, pi, n_k, endpoint=True)
kx, ky = np.meshgrid(k_lin, k_lin)

self_plot = lambda kx, ky: Sigma_kp[:,Idx(k_mesh.locate_neighbours((pi/2,pi/2,0))),:]((kx,ky,0),0).imag

plt.figure(figsize=(8,8))
gs=plt.GridSpec(2,2)
ax = plt.subplot(gs[1],aspect="equal")
plt.pcolormesh(kx, ky, np.vectorize(self_plot)(kx,ky))
plt.colorbar(shrink=0.8)

plt.axes
plt.xlabel(r"$k'_{x}$", fontsize=18)
plt.ylabel(r"$k'_{y}$", fontsize=18)
ax.set_yticks([-np.pi,-np.pi/2.,0,np.pi/2,np.pi])
ax.set_yticklabels([r"$-\pi$",r"$-\frac{\pi}{2}$","0",r"$\frac{\pi}{2}$",r"$\pi$"],fontsize=16)
ax.set_xticks([-np.pi,-np.pi/2.,0,np.pi/2,np.pi])
ax.set_xticklabels([r"$-\pi$",r"$-\frac{\pi}{2}$","0",r"$\frac{\pi}{2}$",r"$\pi$"],fontsize=16)
plt.tight_layout(pad=0,w_pad=2,h_pad=0)
plt.title(r"Im $\tilde{\Sigma}_{\mathbf{k'}}(\mathbf{k}=(\pi/2,\pi/2), i\omega_{0})$",size=18)
plt.show()